# Configuração Sistema Kartado

HALLO ImT! Welcome Again!

Esse script visa gerar automaticamente uma série de [JSONs](https://www.json.org/json-pt.html) utilizados na parametrização do sistema Kartado

São eles:

 - `company.custom_options`
 - `company.metadata`
 - `pemissions`
 - `occurrence_types`
 - `roads`
 
Precisamos também colocar a camada de mapa, que já esta pronta, mas precisa ser associada a company

 - `tile_layers`
 
Existem ainda algumas parâmetrizações que devemos fazer diretamente na

 - Equipes: `firms`
 - Status: `status`
 - Empresa: `sub_company`
 
---

Este código foi desenvolvido por:

 - Leonardo Ramos
 - Felipe Vanroo
 - Pedro Gasparotti

## Bibliotecas

In [37]:

# ---------
# Libraries
# ---------

import json as js
import pandas as pd
import os
import zipfile
import shutil

from os.path import basename
from zipfile import ZipFile

from pathlib import Path
from collections import defaultdict

## Importação de Arquivo

In [38]:
# -----------------------
# DEFINIR NOME DO ARQUIVO
# -----------------------

filename = "CAJ15"
company ="Kartado"

# ---------------------
# DEFINIR OPÇÕES GERAIS
# ---------------------

format_prefix = "{prefixo}-{anoCompleto}.{serialAno:05}" # Configuração da geração do serial
uf = 51 # Código IBGE da uf; Disponível em: https://www.ibge.gov.br/explica/codigos-dos-municipios.php
numFieldsOfArrayInMeasures = 5 #Define o numero de vetores de exportação do array of object

# ----------------
# COMEÇO DE CÓDIGO
# ----------------

name= filename + ".xlsx"
base = Path(filename)

baseForm = Path(filename+"/Formularios")
baseRoad = Path(filename+"/Rodovias")
basePerm = Path(filename+"/Permissoes")
basePerm = Path(filename+"/Unidade")

base.mkdir(exist_ok=True)
baseForm.mkdir(exist_ok=True)
baseRoad.mkdir(exist_ok=True)
basePerm.mkdir(exist_ok=True)

cwd = os.getcwd()
shutil.copy(name, cwd + '\\' + filename)

'C:\\Users\\Guarita\\PyDocs\\CAJ15\\CAJ15.xlsx'

In [39]:
#file = pd.read_excel(name,na_values="",sheet_name=list(range(16)))

# ------------------
# Definição das abas

#capa = pd.read_excel(name,na_values="",sheet_name="CAPA")
# ----
#parametros = pd.read_excel(name,na_values="",sheet_name="Parametros")
#aux = pd.read_excel(name,na_values="",sheet_name="AUX")
# ----
unidade = pd.read_excel(name,na_values="",sheet_name="Informações Gerais")
# ----
permissoes = pd.read_excel(name,na_values="",sheet_name="Permissões")
# ----
rodovias = pd.read_excel(name,na_values="",sheet_name="Rodovias")
# ----
classes = pd.read_excel(name,na_values="",sheet_name="Classes")
formularios = pd.read_excel(name,na_values="",sheet_name="Formularios")
campos = pd.read_excel(name,na_values="",sheet_name="Campos")
# ----
innerfields = pd.read_excel(name,na_values="",sheet_name="innerfields")
options = pd.read_excel(name,na_values="",sheet_name="options")
#---
#correlation = pd.read_excel(name,na_values="",sheet_name="correlation")
# ---
#transporte = pd.read_excel(name,na_values="",sheet_name="Transporte")
#contrato = pd.read_excel(name,na_values="",sheet_name="Contrato")


# ------------
# Visualização

classes.tail()


C:\Users\Guarita\AppData\Local\Programs\Python\Python310\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


,Cor,Classe,id,Frequência,Prazo,id_Natureza,Natureza,Formulario,Transporte,Codigo,CSP,Grupo,OBS
7,NaN,Ocorrências,12,NaN,NaN,5,Ocorrências,occ,NaN,NaN,NaN,NaN,NaN
8,NaN,Avaliações Gerais,13,NaN,NaN,4,Avaliação,faeFacB,NaN,NaN,NaN,NaN,NaN
9,NaN,Pavimentação,15,NaN,NaN,1,Serviços,pav,NaN,NaN,NaN,NaN,NaN
10,NaN,Avaliações v3,16,NaN,NaN,4,Avaliação,avaliacoesV3,NaN,NaN,NaN,NaN,NaN
11,NaN,Adutoras,17,NaN,NaN,2,Inventário,adutora,NaN,NaN,NaN,NaN,NaN


## Definição de funções

Funções para construção de formulários

In [40]:
# ------------------
# Remover Duplicatas
# ------------------

def rd(x):
    """
    Função para remover duplicatas de uma lista
    rd(["a","b","b","a","c","a","b"]) = ["a","b","c"]
    """
    return list(dict.fromkeys(x))

# --------------
# Adciona opções
# --------------

def addOptions(fildObject):
    """
    Adiciona o formField no formData
    Entrada: ["Opção 1","Opção 2","Opção 3"]
    Inicia "value" em 1
    """
    
    op = []
    for value, opt in enumerate(fildObject):                               
        op.append({
            "name": opt,
            "value": str(value+1)
        })
        # ----
    
    return op

# ----------------------
# Adciona opções com ids
# ----------------------

def addOptionsIds(fildObject):
    """
    Adiciona o formField no formData
    Entrada: [["Opção 1","Opção 2","Opção 3"],["6","7","8"]]
    """
    op = []
    for item in range(len(fildObject[0])):
        op.append({
                "name": fildObject[0][item],
                "value": str(fildObject[1][item])
            })
    
    return op

# -----------------------------------------
# Funções utilizadas para occurrences_types
# -----------------------------------------

# Adiciona Fields
# ---------------

def addFields(fildObject):
    """
    Adiciona o formField no formData
    Entrada: [0:displayName,1:apiName,2:dataType,3:jsonLogic,4:autofill,5:displayIf,6:disableIf,7:requiredIf,8:unit,9:mainReport]
    """
    
    field = {
        "displayName" : fildObject[0],
        "apiName" : fildObject[1],
        "dataType" : fildObject[2]
    }
    
    # -----
    # logic
    
    if fildObject[2] == "jsonLogic" or fildObject[2] == "innerLogic":
        try: 
            logic = eval(fildObject[3]) 
        except:
            print('Json Logic wrong in',fildObject[0])
        field.update({"logic":logic})
    
    # --------
    # autofill
    
    if pd.notna(fildObject[4]):
        try: 
            autofill = eval(fildObject[4]) 
        except:
            autofill = str(fildObject[4])
        field.update({"autofill":autofill})
    
    # ---------
    # displayIf
    
    if pd.notna(fildObject[5]):
        try: 
            displayIf = eval(fildObject[5])
        except:
            print("displayIf wrong!")
        field.update({"displayIf":displayIf})
        
    # ---------
    # disableIf
    
    if pd.notna(fildObject[6]):
        try: 
            disableIf = eval(fildObject[6]) 
        except:
            print("disableIf wrong in ",fildObject[0],"! \n disableIf: True Activated")
            disableIf = True
        field.update({"disableIf":disableIf})
    
    # ----------
    # requiredIf
    
    if pd.notna(fildObject[7]):
        try: 
            requiredIf = eval(fildObject[7])
        except:
            print("requiredIf wrong!")
        field.update({"requiredIf":requiredIf})
    
    # -----
    # unit
    
    if pd.notna(fildObject[8]):
        unit = str(fildObject[8])
        field.update({"unit":unit})
        
    # ----------
    # mainReport
    
    if pd.notna(fildObject[9]):
        field.update({"mainReport":True})
        
    return field

# Adiciona MeasuerementCollumns
# -----------------------------

def addMeasures(fildObject):
    """
    Adiciona as exportações a partis de data[fields]
    Entrada: {formField}
    """
    
    model = ["number","float","string","textArea","boolean","timeStamp","timestamp"]
    
    # -----------------------------
    # Padroes fora array_of_Objects
    
    if fildObject["dataType"] != "arrayOfObjects":
            
        header = fildObject["displayName"]
        key = fildObject["apiName"]
        measure = {"key":key,"header":header}
        
        if fildObject["dataType"] in model:
            measure.update({"logic":{"var":"formData."+fildObject["apiName"]}})

        if fildObject["dataType"] == "jsonLogic":
            measure.update({"logic":fildObject["logic"]})

        if fildObject["dataType"] == "timestamp":
            measure.update({"isDate":True})

        if fildObject["dataType"] == "select":

            if_select = []
            for option in fildObject["selectOptions"]["options"]:
                
                select_logic = {"==":[{"var":str("formData."+fildObject["apiName"])},option["value"]]}
                if_select.append(select_logic)
                if_select.append(option["name"])

            if_select.append("")

            measure.update({"logic":{"if":if_select}})

        if fildObject["dataType"] == "selectMultiple":

            catM = []

            for option in fildObject["selectOptions"]["options"]:
                
                selectM_logic = {"in":[str(option["value"]),{"var":str("formData."+fildObject["apiName"])}]}
                if_selectMObject = {"if": [selectM_logic,str(option["name"]+"; "),str("")]}
                catM.append(if_selectMObject)
                
            measure.update({"logic":{"cat":catM}})
            
    # ----------------
    # Array_of_objects
    
    if fildObject["dataType"] == "arrayOfObjects":
        
        measure = []
        
        for i in range(numFieldsOfArrayInMeasures):
            for inner in fildObject["innerFields"]:
                
                header = inner["displayName"]+" - "+str(i+1)
                key = inner["apiName"]+str(i+1)
                measure_inner = {"key":key,"header":header}
                
                if inner["dataType"] in model:
                    measure_inner.update({"logic":{"var":"formData."+fildObject["apiName"]+"."+str(i)+"."+inner["apiName"]}})
                
                if inner["dataType"] == "timestamp":
                    measure_inner.update({"isDate":True})
                    
                if inner["dataType"] == "innerImagesArray":
                    measure_inner.update({"logic":{"var":"formData."+fildObject["apiName"]+"."+str(i)+"."+inner["apiName"]}})
                    measure_inner.update({"isImage":True})
                    measure_inner.update({"repeatInTheEnd":False})
                    measure_inner.update({"numberOfImages":3})
                
                if inner["dataType"] == "select":

                    if_select = []

                    for option in inner["selectOptions"]["options"]:

                        select_logic = {"==":[{"var":"formData."+fildObject["apiName"]+"."+str(i)+"."+inner["apiName"]},option["value"]]}
                        if_select.append(select_logic)
                        if_select.append(option["name"])

                    if_select.append("")

                    measure_inner.update({"logic":{"if":if_select}})

                if inner["dataType"] == "selectMultiple":
                    
                    catM = []

                    for option in inner["selectOptions"]["options"]:
                        
                        selectM_logic = {"in":[option["value"],{"var":"formData."+fildObject["apiName"]+"."+str(i)+"."+inner["apiName"]}]}
                        if_selectMObject = {"if":[selectM_logic,str(option["name"]),str("")]}
                        catM.append(if_selectMObject)
                        
                    measure_inner.update({"logic":{"cat":catM}})
            
                measure.append(measure_inner)
            
    return measure

# ---------------------------------
# Funções utilizadas em Permissions
# ---------------------------------

# Compara permissões
# ---------------------------

def compare(xm,ym):
    """
    Comparação entre permissões
    Entrada: {permissions}
    """
    comum = []
    exception = []
    
    for key in xm.keys():
        if (key not in comum) and (key not in exception):
            if key in ym.keys():
                comum.append(key)
            else:
                exception.append(key)
        else:
            pass
        
    for key in ym.keys():
        if (key not in comum) and (key not in exception):
            if key in xm.keys():
                comum.append(key)
            else:
                exception.append(key)
        else:
            pass
        
    resp = {"comum":comum, "exception":exception}
    
    return resp

## Configuração de formulários

### Configurações gerais 
 
 > Para não mostrar no app `"displayIf": {"!==":[{"var": "isMobile"}, True]}`
 
 > Para manter apontamento como obrigatório `"requiredIf": { "!": { "var": "value" } }`
 
 > Funções interessantes: https://jsonlogic.com/operations.html
 
 > Função de arrendondamento `"round":[[{formdata},0]]` em autofill

### Configuração de options

 > Padrao iniciar com "value":1

### Configuração de innerlogic

 > O innerlogic não será calculado!

 > Para desabilitar a edição, é considerado inner innerlogic = autofill + disableIf (true)

 > `"var":[{"cat": ["formData.array.", index ,".step"]}]` para acesso da info na exportação

In [41]:
# ---
# Exportação do Arquivo

with ZipFile(filename + '.zip', 'w', zipfile.ZIP_DEFLATED) as zipObj:
    for folderName, subfolders, files in os.walk(cwd + '\\' + filename):
        for file in files:
            filePath=os.path.join(folderName, file)
            zipObj.write(filePath, basename(filePath))
            
shutil.move(cwd + '\\' + filename + '.zip', cwd + '\\' + filename + '\\' + filename + '.zip')

'C:\\Users\\Guarita\\PyDocs\\CAJ15\\CAJ15.zip'

In [42]:
# --------
# Exporter
# --------

exporter_keys = []
exporter = []

for row in classes.iterrows():
    
    data={} 
    data["id"] = row[1]["id"]
    data["name"] = str(row[1]["Classe"])
    data["displayName"] = str(row[1]["Classe"])
    data["fields"] = []
    data["groups"] = []
    data["measurementColumns"] = []
    
    num_field = 1
    num_order=1
    
    print("\n\n",str(row[1]["Classe"]))
    
    # Inicia foramção dos grupos
    
    grupos = defaultdict(list)
    
    # ----------------
    # Campos especiais
    # ----------------
    """
    Campos especiais diferenciam-se por classe, ex: Código de edital, csp...
    Neste caso precisamos cadastrar seus campos na mão
    Podemos automatizar esse processo no futuro
    """
    
    if False : # Inserir opção de coluna como pd.notNa 
        members = []
        
        data["fields"].append({  
            "id": num_field,
            "apiName": "infringement",
            "autofill": str(row[1]["Infração"]),
            "disableIf":True,
            "dataType": "string",
            "displayName": "Infração",
        })
        members.append(num_field)
        num_field+=1
        
        data["measurementColumns"].append({  
            "key": "infringement",
            "logic": str(row[1]["Infração"]),
            "width": 12,
            "header": "Infração"
        })
        
        data["groups"].append({
            "order": num_order,
            "members": members,
            "displayName": "Informações de Infração"
        })
        num_order+=1
    
    # -------------
    # Campos cadastrados   
    
    fields = formularios[formularios["Formulario"] == row[1]["Formulario"]]
    
    for index, field in fields.iterrows():
        
        settingField = campos[campos["apiName"] == field["apiName"]]
        settingFieldList = settingField.values.tolist()
        
        if len(settingFieldList) == 0:
            print("WAAAARNING!\n",field[1],"não esta definido! \n")
        
        for i in settingFieldList:
            
            formField = addFields(i[0:10])
            formField.update({"id":num_field})
            
            # ------------------------
            # select or selectMultiple
            
            if i[2] == "select" or i[2] == "selectMultiple":
                
                optionsBoolean = pd.notna(options[i[1]])
                selectOptions = addOptions(options[i[1]][optionsBoolean])
                
                formField.update({"selectOptions": {
                    "options": selectOptions
                }})
                
            # -------------
            # arrayOfObject
            
            if i[2] == "arrayOfObjects":
                
                settingInnerField = innerfields[innerfields["apiName"] == i[1]]
                settingInnerFieldList = settingInnerField.values.tolist()
                
                num_inner = 1
                innerFields = []
                
                for j in settingInnerFieldList:
                
                    innerField = addFields(j[2:12])
                    innerField.update({"order":num_inner})
                    num_inner += 1
                
                    # ------------------------
                    # select or selectMultiple
                
                    if j[4] == "select" or j[4] == "selectMultiple":

                        optionsBoolean = pd.notna(options[j[3]])
                        selectOptions = addOptions(options[j[3]][optionsBoolean])

                        innerField.update({"selectOptions": {
                            "options": selectOptions
                        }})
                
                    
                    innerFields.append(innerField)
                
                formField.update({"innerFields": innerFields})
                
            # ---------------------------
            # Adição do campo no formdata
            
            data["fields"].append(formField)
            
            # -------------------------------
            # Adição das exportações formdata
            
            measurementColumn = addMeasures(formField)
            
            if type(measurementColumn) == list:
                
                data["measurementColumns"].extend(measurementColumn)
            
                for a in measurementColumn:
                    
                    if a["key"] not in exporter_keys:
                        exporter_keys.append(a["key"])
                        exporter.append(a)
            else:
                
                data["measurementColumns"].append(measurementColumn)
                
                if measurementColumn["key"] not in exporter_keys:
                    exporter_keys.append(measurementColumn["key"])
                    exporter.append(measurementColumn)
            
            # ---------------
            # Adição de grupo
        
            grupos[field["Grupo"]].append(num_field)
            num_field += 1
        
    # -------------------   
    # Montagem dos grupos
    
    grupos=dict(grupos)
    
    for key, grupo in grupos.items():
        
        data["groups"].append({
            "order": num_order,
            "members": grupo,
            "displayName": key,
        })
        num_order += 1
        
    # -------------------------------------
    # Preparação do arquivo para exportação
    # -------------------------------------
        
    with open(filename + '/Formularios/' + str(row[1]["id"])+".txt", "w") as outfile:  
        js.dump(data, outfile, indent=4)




 Ordem de Serviço


 Execução de serviço


 Execução de Rede VCA


 Execução de Rede MND


 Execução de Pavimentação


 Execução de Poliedros/Intertravados


 Execução de Pavimento em concreto


 Ocorrências


 Avaliações Gerais


 Pavimentação


 Avaliações v3


 Adutoras


In [43]:
# Conferencia da exportação a ser inserida em no custom_options da company para Excel Simplificado

print(exporter)

# -------------------------------------
# Preparação do arquivo para exportação
# -------------------------------------
        
with open(filename + '/' +"exporter.txt", "w") as outfile:  
        js.dump(exporter, outfile, indent=4)

# Conferencia do ultimo form
#data

[{'key': 'basin', 'header': 'Bacia', 'logic': {'var': 'formData.basin'}}, {'key': 'subbasin', 'header': 'Sub Bacia', 'logic': {'var': 'formData.subbasin'}}, {'key': 'place', 'header': 'Nome do Logradouro', 'logic': {'var': 'formData.place'}}, {'key': 'stretch', 'header': 'Número do Trecho', 'logic': {'var': 'formData.stretch'}}, {'key': 'lengthStretch', 'header': 'Extensão Trecho (m)', 'logic': {'var': 'formData.lengthStretch'}}, {'key': 'slope', 'header': 'Declividade do Trecho (m/m)', 'logic': {'var': 'formData.slope'}}, {'key': 'diameterColector', 'header': 'Diâmetro do Coletor (mm)', 'logic': {'var': 'formData.diameterColector'}}, {'key': 'pavement', 'header': 'Tipo de Pavimento', 'logic': {'var': 'formData.pavement'}}, {'key': 'pvNumber1', 'header': 'Número do PV - 1', 'logic': {'var': 'formData.manhole.0.pvNumber'}}, {'key': 'pvHeight1', 'header': 'Cota do Terreno (m) - 1', 'logic': {'var': 'formData.manhole.0.pvHeight'}}, {'key': 'pvElevation1', 'header': 'Cota de Fundo do Colet

## Configuração da Company

In [28]:
# -----------------------------
# ATENÇÃO: VERIFICAÇÕES MANUAIS
# -----------------------------

# ------
# Faixas
# ------

faixa_linha_inicial = 42
faixa_linha_final = 53

# ------
# Sentido
# ------

sentido_linha_inicial = 69
sentido_linha_final = 74

# ---------
# Naturezas
# ---------

natureza_linha_inicial = 78
natureza_linha_final = 86

# -----
# Lotes
# -----

lote_linha_inicial = 97
lote_linha_final = 101

In [30]:
# ------------------------
# Configurações de company
# ------------------------

# Importação do 
with open("Base\\Unidade\\custom_options.txt", 'r') as co:  
    custom_options = js.load(co)

In [33]:
naturezas = unidade.iloc[(natureza_linha_inicial-2):(natureza_linha_final-1),2:5]
naturezas.columns = ["Natureza","id_Natureza","pref_Natureza"]

natureza_option = addOptionsIds([list(naturezas["Natureza"]),list(naturezas["id_Natureza"])])
#print(natureza_option)

custom_options["reporting"]["fields"]["occurrence_kind"]["select_options"]["options"]=natureza_option
custom_options["occurrence_type"]["fields"]["occurrence_kind"]["select_options"]["options"]=natureza_option
custom_options["reporting_deadline"]["fields"]["occurrence_kind"]["select_options"]["options"]=natureza_option

In [34]:
faixas = unidade.iloc[(faixa_linha_inicial-2):(faixa_linha_final-1),3:5]
faixas.columns = ["Faixas","id_Faixas"]

faixas_option = addOptionsIds([list(faixas["Faixas"]),list(faixas["id_Faixas"])])
#print(faixas_option)

custom_options["reporting"]["fields"]["lane"]["select_options"]["options"]=faixas_option
custom_options["occurrence_type"]["fields"]["lane"]["select_options"]["options"]=faixas_option
custom_options["reporting_deadline"]["fields"]["lane"]["select_options"]["options"]=faixas_option

In [35]:
sentidos = unidade.iloc[(sentido_linha_inicial-2):(sentido_linha_final-1),3:5]
sentidos.columns = ["Sentido","id_Sentido"]

sentidos_option = addOptionsIds([list(sentidos["Sentido"]),list(sentidos["id_Sentido"])])
#print(faixas_option)

custom_options["reporting"]["fields"]["direction"]["select_options"]["options"]=sentidos_option
custom_options["occurrence_type"]["fields"]["direction"]["select_options"]["options"]=sentidos_option
custom_options["reporting_deadline"]["fields"]["direction"]["select_options"]["options"]=sentidos_option

In [36]:
print(custom_options)

{'user': {'fields': {'metadata': {'role': {'display_if': {'logic': True}}, 'occupation': {'display_if': {'logic': True}}, 'board_registration': {'display_if': {'logic': True}}}, 'create_access_request': {'display_if': {'logic': True}}}}, 'inventory': {'filters': {}, 'list_columns': []}, 'reporting': {'fields': {'lane': {'api_name': 'lane', 'data_type': 'select', 'display_name': 'Faixa', 'select_options': {'options': [{'name': 'Faixa 1', 'value': '0'}, {'name': 'Faixa 2', 'value': '1'}, {'name': 'Acostamento', 'value': '2'}, {'name': 'Faixa 3', 'value': '3'}, {'name': 'Ponte', 'value': '4'}, {'name': 'Viaduto', 'value': '5'}, {'name': 'Tunel', 'value': '6'}, {'name': 'Faixa de aceleração', 'value': '7'}, {'name': 'Faixa de desaceleração', 'value': '8'}, {'name': 'Faixa adicional', 'value': '9'}, {'name': 'Faixa de domínio', 'value': '10'}, {'name': 'Pedágio', 'value': '11'}]}}, 'direction': {'api_name': 'direction', 'data_type': 'select', 'display_name': 'Sentido', 'select_options': {'o

In [138]:
lotes = unidade.iloc[(lote_linha_inicial-2):(lote_linha_final-1),3:5]
lotes.columns = ["Lotes","id_Lotes"]

lotes_option = addOptionsIds([list(lotes["Lotes"]),list(lotes["id_Lotes"])])
lotes_option

[{'name': 'Executado', 'value': '4'},
 {'name': 'Retrabalho', 'value': '5'},
 {'name': nan, 'value': 'nan'},
 {'name': nan, 'value': 'nan'},
 {'name': 'Demais configurações', 'value': 'nan'},
 {'name': nan, 'value': 'nan'},
 {'name': 'A aba "Usuários" contém uma tabela com as informações necessárias para inclusão de usuários no sistema.\n\nAs senhas serão enviadas por email',
  'value': 'nan'},
 {'name': nan, 'value': 'nan'},
 {'name': 'A aba de "Rodovias" contém uma tabela com as informações necessárias para inclusão de rodovias no sistema.\n\nAceitamos também traçados em .kmz/.kml',
  'value': 'nan'},
 {'name': nan, 'value': 'nan'},
 {'name': 'A aba de "Classes" contém uma tabela com a árvore de serviços do sistema.',
  'value': 'nan'}]

In [37]:
serial_format = defaultdict(int)

for item in naturezas.iterrows():
    serial_format[str(item[1]["id_Natureza"])] = {"type":str(item[1]["pref_Natureza"]),"format":str(format_prefix)}
    
serial_format["default"] = {"type":"default","format":str(format_prefix)}
serial_format = dict(serial_format)
serial_format

{'1': {'type': 'Lev', 'format': '{prefixo}-{anoCompleto}.{serialAno:05}'},
 '2': {'type': 'Inv', 'format': '{prefixo}-{anoCompleto}.{serialAno:05}'},
 '5': {'type': 'Dia', 'format': '{prefixo}-{anoCompleto}.{serialAno:05}'},
 '6': {'type': 'Pav', 'format': '{prefixo}-{anoCompleto}.{serialAno:05}'},
 '7': {'type': 'Con', 'format': '{prefixo}-{anoCompleto}.{serialAno:05}'},
 '8': {'type': 'Dre', 'format': '{prefixo}-{anoCompleto}.{serialAno:05}'},
 '9': {'type': 'Dis', 'format': '{prefixo}-{anoCompleto}.{serialAno:05}'},
 '10': {'type': 'Sin', 'format': '{prefixo}-{anoCompleto}.{serialAno:05}'},
 '11': {'type': 'Tra', 'format': '{prefixo}-{anoCompleto}.{serialAno:05}'},
 'default': {'type': 'default',
  'format': '{prefixo}-{anoCompleto}.{serialAno:05}'}}

## Configuração das Permissoes

In [8]:
# --- Readind excel ---

# ---
# INSERIR O NOME DA UNIDADE AQUI!

#name="CAJ.xlsx" #Se mesmo name acima, colocar como comentário
#permissoes = pd.read_excel(name,sheet_name='Permissões', na_values='')

# ---
# EXIBIÇÃO

traducao = {
    "Todos" : "all",
    "Apenas da sua equipe" : "firm",
    "Apenas os que criou" : "self",
    "Padrão" : "default",
    "Condição especial" : "ESPECIAL",
    "Nenhum" : "none",
    "Sim":True,
    "Não":False
}


In [9]:



passo = 8
numero_de_permissoes = 7

for i in range(7):
    
    # -------------------------
    # Iteração entre Permissoes
    
    i_i = 0
    i_f = 6
    j_i = 1
    j_f = 20

    i_i = i_i + (passo*i)
    i_f = i_f + (passo*i)
    j_i = j_i 
    j_f = j_f 

    print(i_i,i_f)
    
    # -------------------
    # Para cada permissao

    permissao = permissoes.iloc[i_i:i_f,j_i:j_f]
    print(permissao)

    permission = {}

    for chave in chaves[(chaves["Geral"].notna()) & (chaves["Especial"].isnull())]["Chaves de permissões"]:

        #print(chave)

        permission.update({str(chave):{
            "can_view":True,
            "can_edit":True,
            "queryset":"all",
            "can_create":True,
            "can_delete":False
        }})

    for row in chaves[(chaves["Geral"].notna()) & (chaves["Especial"].notna())].iterrows():

        #print(row[1])

        permission.update({str(row[1]["Chaves de permissões"]):{
            str(row[1]["Especial"]):True,
        }})

    #print(permissao)

    for column in permissao:
        #print(chaves[chaves[permissao[column].name].notna()][["Chaves de permissões"]])

        permissionsSet = chaves[chaves[permissao[column].name].notna()]
        #print(type(permissionsSet))

        for permissionSet in permissionsSet.iterrows():
            #print(permissionSet[1])
            print(permissao[column].name,"Define a chave",permissionSet[1]["Chaves de permissões"])

            listPermisson = []
            for item in permissao[column]:
                listPermisson.append(item)
            #print(listPermisson)

            permissionStep={str(permissionSet[1]["Chaves de permissões"]) : {
                "can_view": traducao[listPermisson[0]],
                "can_edit": traducao[listPermisson[1]],
                "queryset": traducao[listPermisson[4]],
                "can_create": traducao[listPermisson[2]],
                "can_delete": traducao[listPermisson[3]]
            }}
            #print(permissionStep)

            # ---
            # Can approve
            if pd.notna(listPermisson[5]):
                permissionStep[str(permissionSet[1]["Chaves de permissões"])].update({"can_approve":traducao[listPermisson[5]]})

            # ---
            # Especiais acrescentadas

            if pd.notna(permissionSet[1]["Especial"]):

                permissionSet[1]["Especial"]=permissionSet[1]["Especial"].replace("\n","")
                especialItens =  permissionSet[1]["Especial"].split(";")

                for item in especialItens:
                    permissionStep[str(permissionSet[1]["Chaves de permissões"])].update({str(item):True})

            permission.update(permissionStep)

    #print(permission)

    with open(filename + '/Permissoes/' + str(i)+'.txt', 'w') as outfile:  
        js.dump(permission, outfile, indent=4)

0 6
  Apontamentos PainelPazos Inventário Formulários Programações Objetos  \
0          Sim         Sim        Sim         Sim          Sim     Sim   
1          Sim         Sim        Sim         Sim          Sim     Sim   
2          Sim         Sim        Sim         Sim          Sim     Sim   
3          Sim         Sim        Sim         Sim          Sim     Sim   
4        Todos       Todos      Todos       Todos        Todos   Todos   
5          Sim         NaN        NaN         NaN          NaN     NaN   

  Recursos provisionados Recursos utilizados Equipes Usuários  \
0                    Sim                 Sim     Sim      Sim   
1                    Sim                 Sim     Sim      Sim   
2                    Sim                 Sim     Sim      Sim   
3                    Sim                 Sim     Sim      Sim   
4                  Todos               Todos   Todos    Todos   
5                    NaN                 Sim     NaN      NaN   

  Solicitações de ace

### Permissões v2

A partir do Base dos módulos basicos de montagem

In [5]:
moduloB = "P_Basico"
moduloI = "P_Intermediario"
moduloA = "P_Avancado"
moduloC = "P_Avancado2"

with open("AA - Permissoes\\" + moduloB + ".txt", 'r') as x1:  
    b = js.load(x1)

with open("AA - Permissoes\\" +moduloI+".txt", 'r') as x2:  
    i = js.load(x2)
    
with open("AA - Permissoes\\" +moduloA+".txt", 'r') as x3:  
    a = js.load(x3)
    
with open("AA - Permissoes\\" +moduloC+".txt", 'r') as x4:  
    c = js.load(x4)

In [8]:
exclude = []
muduloAnalises = c

for key in muduloAnalises.keys():
    try:
        if muduloAnalises[key]["queryset"]=="none":
            print(key," retirada")
            exclude.append(key)
    except:
        print(key," sem query")
        
#for item in exclude:
#    muduloAnalises.pop(item)

NameError: name 'permissao_teste' is not defined

#### __UTILIZAR FUNÇÃO COMPARE PARA COMPARAR PERMISSIONS DISTINTAS__

Olhar `def compare(permission1,permission2)`

## Configuração das Rodovias

In [9]:
# --- Readind excel ---

# ---
# EXIBIÇÃO 
rodovias.tail()

,Rodovia,roadName,Lote,km,latitude (norte),longitude (leste),Cresc.(1)/Dec.(2),Sentido,id_Sentido,mapboxCall,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14
0,SP-294,NaN,Lote1,0,-22.228305,-49.906477,1,Norte,0,x,NaN,NaN,"-22.2283046,-49.906477",-22.228305,-49.906477
1,SP-294,NaN,Lote1,1,-22.230287,-49.916135,1,Norte,0,x,NaN,NaN,"-22.2302867,-49.916135",-22.230287,-49.916135


In [11]:
listaRoadName = rd(rodovias['Rodovia'].replace("/","-"))

for i in listaRoadName:
    
    ind = 0
    marks = {}
    jason = {}
    
    #print(i)
    
    for row in rodovias[rodovias['Rodovia'].replace("/","-")==i].iterrows():
        
        #print(row[1])

        coordinates=[]
        coordinates.append(row[1]["longitude (leste)"])
        coordinates.append(row[1]["latitude (norte)"])

        point = {
            'type':'Point',
            'coordinates': coordinates
        }
        
        mark = {
            'km': row[1]["km"],
            'point': point
        }
        
        if pd.notna(row[1]["mapboxCall"]):
            mark.update({"mapbox_call":True})
       
        marks.update({
            str(ind) : mark
        })
        ind+=1
        
        #print(ind)
        
        name = row[1]["Rodovia"]
        direc = row[1]["Sentido"]
        cresc = row[1]["Cresc.(1)/Dec.(2)"]
        codig = row[1]["id_Sentido"]
        
        if cresc == 1:
            if codig == 0:
                metadata = {
                    "rising": "0",
                    "falling": "1",
                    "values" :  ["0", "1", "4", "5"]
                }

            if codig == 1:
                metadata = {
                    "rising": "1",
                    "falling": "0",
                    "values" :  ["0", "1", "4", "5"]
                }

            if codig == 2:
                metadata = {
                    "rising": "2",
                    "falling": "3",
                    "values" :  ["2", "3", "4", "5"]
                }

            if codig == 3:
                metadata = {
                    "rising": "3",
                    "falling": "2",
                    "values" :  ["2", "3", "4", "5"]
                }

        elif cresc == 2:
            if codig == 0:
                metadata = {
                    "rising": "1",
                    "falling": "0",
                    "values" :  ["0", "1", "4", "5"]
                }

            if codig == 1:
                metadata = {
                    "rising": "0",
                    "falling": "1",
                    "values" :  ["0", "1", "4", "5"]
                }

            if codig == 2:
                metadata = {
                    "rising": "3",
                    "falling": "2",
                    "values" :  ["2", "3", "4", "5"]
                }

            if codig == 3:
                metadata = {
                    "rising": "2",
                    "falling": "3",
                    "values" :  ["2", "3", "4", "5"]
                }
        else:
            
            metadata=None
        
    #print(name)    
    
    jason.update({
        "name" : name,
        "direction" : direc,
        "marks" : marks,
        #"uf" : uf,
        "metadata": metadata,
        "company" : company
    })
        
    with open(filename + '/Rodovias/' + str(i)+'.txt', 'w') as outfile:  
        js.dump(jason, outfile, indent=4)

### Lotes

In [118]:
lotes = unidade.iloc[(lote_linha_inicial-2):(lote_linha_final-1),3:5]
lotes.columns = ["Lotes","id_Lotes"]

lotes_option = addOptionsIds([list(lotes["Lotes"]),list(lotes["id_Lotes"])])
lotes_option

lookup_dict = {
    a["name"]: a["value"] for a in lotes_option
}

In [119]:
listaRod = rd(rodovias['Rodovia'])
listaLotes = rd(rodovias['Lote'])

# OBS:
# Não pode haver lotes com trechos descontinuos na mesma rodovia. Ex: Lote A - B - A

lotes = defaultdict(list)

for rodovia in listaRod:
    
    lotLogicLote = []
        
    for lote in listaLotes:
        
        listaKms = rodovias[(rodovias["Lote"]==lote) & (rodovias['Rodovia']==rodovia)]["km"]
        
        try: 
            
            kmMinimo = min(list(listaKms))
            kmMaximo = max(list(listaKms))
            
        except:
            
            continue
             
        else:
            
            # montagem do logico
            
            lotLogic = [
                {"and":[
                    {"!==": [{"var": "data.km"}, None]},
                    {">=": [{"var": "data.km"}, kmMinimo]},
                    {"<=": [{"var": "data.km"}, kmMaximo]}
                ]},
            str(lookup_dict[lote])]
            
            lotLogicLote.extend(lotLogic)
          
    lotLogicLote.append("")
    lotLogicLote = {"if":lotLogicLote} 
    
    lotes[rodovia].append(lotLogicLote)

lotes=dict(lotes)
    
with open(filename + '/Rodovias/Lotes.txt', 'w') as outfile:  
    js.dump(lotes, outfile, indent=4)

# Auxiliares

## Lotes

In [5]:
#print(correlation)

lista_road = rd(correlation["BR"]) 

if1 = []

for road in lista_road:

    if1.append({"==":[{"var":"roadName"},str(road)]})
    if2 = []
    
    for row in correlation[correlation["BR"] == road].iterrows():
        
        if2.append({"and": [{
            ">=": [{"var": "km"}, str(row[1]["km inicial"])]
        }, {
            "<": [{"var": "km"}, str(row[1]["km final"])]
        }]})
        if2.append(str(row[1]["CIDADES"]))
        
    if2.append(str(""))
    if1.append({"if": if2})
    
if1.append(str(""))
        
print({"if":if1})

NameError: name 'rd' is not defined

## Correlação 1:1

In [6]:
correlation

,originDestiny,materialProvider,materialDestiny,valuePerAxis,Dmt,valueCoef,material,precoUnitario,plate,numberOfAxis
0,PEDREIRA SIQUEIRA PARAGUAÇU PAULISTA → SP 270...,PEDREIRA SIQUEIRA PARAGUAÇU PAULISTA,SP 270 KM 451 - ASSIS/SP,16.0,32.0,0.28,AREIA DE BRITA - PARAGUAÇU PAULISTA,40.5,CKU-0620,12
1,PEDREIRA EXPRESSA → SP 270 KM 451 - ASSIS/SP,PEDREIRA EXPRESSA,SP 270 KM 451 - ASSIS/SP,9.6,156.0,0.32,PÓ DE PEDRA - PARAGUAÇU PAULISTA,40.5,CLU-0F13,8
2,PEDREIRA CASTILHO → SP 270 KM 451 - ASSIS/SP,PEDREIRA CASTILHO,SP 270 KM 451 - ASSIS/SP,9.3,120.0,0.32,PEDRISCO - PARAGUAÇU PAULISTA,40.5,EJV-9E10,9
3,PEDREIRA SIQUEIRA PARAGUAÇU PAULISTA → SP 270...,PEDREIRA SIQUEIRA PARAGUAÇU PAULISTA,SP 270 KM 480 - MARACAÍ/SP,0.0,36.0,0.28,PÓ DE PEDRA - CASTILHO,30.0,ELC-0D79,9
4,PEDREIRA EXPRESSA → SP 270 KM 480 - MARACAÍ/SP,PEDREIRA EXPRESSA,SP 270 KM 480 - MARACAÍ/SP,9.3,157.0,0.32,PÓ DE PEDRA - EXPRESSA,32.7,FCD-8J77,11
5,PEDREIRA CASTILHO → SP 270 KM 480 - MARACAÍ/SP,PEDREIRA CASTILHO,SP 270 KM 480 - MARACAÍ/SP,19.6,146.0,0.32,NaN,NaN,FHU-8159,12
6,SP 270 KM 451 - ASSIS/SP → SP 270 KM 480 - MAR...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FPR-IG88,9
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FRJ-4J87,12
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,IPZ-2J07,8


In [ ]:
#print(correlation)

#lista_correlation = correlation[["originDestiny","valuePerAxis","Dmt","valueCoef"]].dropna()
lista_correlation = correlation[["plate","numberOfAxis"]].dropna()

api_origem = "plate"
api_destiny = "numberOfAxis"

lista_correlation
if_condition = []

for item1 in lista_correlation.iterrows():
    if_condition.append({"==":[{"var":"formData."+api_origem},item[0]+1]})
    if_condition.append(item[1][api_destiny])

if_condition.append("")
print({"if":if_condition})
    

## Correlação dupla

In [13]:
correlation

,originDestiny,materialProvider,materialDestiny,valuePerAxis,Dmt,valueCoef,material,precoUnitario,plate,numberOfAxis
0,PEDREIRA SIQUEIRA PARAGUAÇU PAULISTA → SP 270...,PEDREIRA SIQUEIRA PARAGUAÇU PAULISTA,SP 270 KM 451 - ASSIS/SP,16.0,32.0,0.28,AREIA DE BRITA - PARAGUAÇU PAULISTA,40.5,CKU-0620,12
1,PEDREIRA EXPRESSA → SP 270 KM 451 - ASSIS/SP,PEDREIRA EXPRESSA,SP 270 KM 451 - ASSIS/SP,9.6,156.0,0.32,PÓ DE PEDRA - PARAGUAÇU PAULISTA,40.5,CLU-0F13,8
2,PEDREIRA CASTILHO → SP 270 KM 451 - ASSIS/SP,PEDREIRA CASTILHO,SP 270 KM 451 - ASSIS/SP,9.3,120.0,0.32,PEDRISCO - PARAGUAÇU PAULISTA,40.5,EJV-9E10,9
3,PEDREIRA SIQUEIRA PARAGUAÇU PAULISTA → SP 270...,PEDREIRA SIQUEIRA PARAGUAÇU PAULISTA,SP 270 KM 480 - MARACAÍ/SP,0.0,36.0,0.28,PÓ DE PEDRA - CASTILHO,30.0,ELC-0D79,9
4,PEDREIRA EXPRESSA → SP 270 KM 480 - MARACAÍ/SP,PEDREIRA EXPRESSA,SP 270 KM 480 - MARACAÍ/SP,9.3,157.0,0.32,PÓ DE PEDRA - EXPRESSA,32.7,FCD-8J77,11
5,PEDREIRA CASTILHO → SP 270 KM 480 - MARACAÍ/SP,PEDREIRA CASTILHO,SP 270 KM 480 - MARACAÍ/SP,19.6,146.0,0.32,NaN,NaN,FHU-8159,12
6,SP 270 KM 451 - ASSIS/SP → SP 270 KM 480 - MAR...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FPR-IG88,9
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FRJ-4J87,12
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,IPZ-2J07,8


In [46]:
lista_materialProvider = rd(correlation["materialProvider"].dropna())
lista_materialDestiny = rd(correlation["materialDestiny"].dropna())

print(lista_materialDestiny)

['SP 270 KM 451 - ASSIS/SP', 'SP 270 KM 480 - MARACAÍ/SP']


In [53]:
ii=1
item=0
logic_if=[]

api_destiny = "valueCoef"

for i in lista_materialDestiny:
    jj=1
    #print("materialDestiny : ",ii)
    for j in lista_materialProvider:
        #print("materialProvider : ",jj)
        logic_if.append({"and":[
            {"==":[{"var":"formData.materialDestiny"},ii]},{"==":[{"var":"formData.materialProvider"},jj]}
        ]})
        logic_if.append(correlation[api_destiny][item])
        jj +=1
        item +=1
    ii+=1
logic_if.append("")
    

{"if":logic_if}

{'if': [{'and': [{'==': [{'var': 'formData.materialDestiny'}, 1]},
    {'==': [{'var': 'formData.materialProvider'}, 1]}]},
  0.32,
  {'and': [{'==': [{'var': 'formData.materialDestiny'}, 1]},
    {'==': [{'var': 'formData.materialProvider'}, 2]}]},
  0.27,
  {'and': [{'==': [{'var': 'formData.materialDestiny'}, 1]},
    {'==': [{'var': 'formData.materialProvider'}, 3]}]},
  0.27,
  {'and': [{'==': [{'var': 'formData.materialDestiny'}, 2]},
    {'==': [{'var': 'formData.materialProvider'}, 1]}]},
  0.32,
  {'and': [{'==': [{'var': 'formData.materialDestiny'}, 2]},
    {'==': [{'var': 'formData.materialProvider'}, 2]}]},
  0.27,
  {'and': [{'==': [{'var': 'formData.materialDestiny'}, 2]},
    {'==': [{'var': 'formData.materialProvider'}, 3]}]},
  0.27,
  '']}

## Auxiliar controle de exportação

In [46]:
teste = ['buildId','initialStake','initialFraction','finalStake','finalFraction','length','volumeDescritivo','notes','amount','viatura','plate','initialHour','route','finalHour','material','tonnage','diesel','gasolina','etanol','oleo','oleo1540','oleo8590','graxa','horimetroInitial','horimetroFinal','horimetro','travels','designacao','marca','modelo','serie','anoFabricacao','checkA','checkB','checkC','checkD','checkE','checkF','checkG','checkH','checkI','checkJ','checkK','checkL','checkM','checkN','checkO','checkP','checkQ','avaliationType','checkOne','checkTwo','checkThree','checkFour','checkFive','checkSix','checkSeven','checkEight','checkNine','checkOneZero','checkOneOne','checkOneTwo','checkOneThree','checkOneFour','checkOneFive','checkOneSix','checkOneSeven','checkOneEight','checkOneNine','checkTwoZero','checkTwoOne','checkTwoTwo','checkTwoThree','checkTwoFour','checkTwoFive','checkTwoSix','checkTwoSeven','checkTwoEight','checkTwoNine','checkThreeZero','checkThreeOne','checkThreeTwo','checkThreeThree','checkThreeFour','checkThreeFive','checkThreeSix','checkThreeSeven','checkThreeEight','checkThreeNine','checkFourZero','checkFourOne','checkFourTwo','checkFourThree','checkFourFour','checkFourFive','checkFourSix','checkFourSeven','checkFourEight','checkFourNine','checkFiveZero','checkFiveOne','checkFiveTwo','checkFiveThree','checkFiveFour','checkFiveFive','formCode','previewDate']

extra_collumns = exporter

In [50]:
export = []
export2 = []

for item in extra_collumns:
    print(item["header"])
    #if item["header"] in export2:
    #    print(item["header"])
    #else:
    export.append(item["key"])
    export2.append(item["header"])
    

Identificação da Obra
Estaca Inicial - Menor
Fração Inicial (m)
Estaca Final - Maior
Fração Final (m)
Comprimento
Volume
Observações
Quantidade
Viatura
Placa
Hora Inicio
Percurso
Hora Fim
Material transportado
Peso
Diesel (L)
Gasolina (L)
Etanol (L)
Óleo 68 (L)
15W40 (L)
85W90 (L)
Graxa (kg)
Horímetro Inicial
Horímetro Final
Horímetro
Hora Inicio - 1
Km origem - 1
Percurso - 1
Hora Fim - 1
Km fim - 1
Material transportado - 1
Peso - 1
Hora Inicio - 2
Km origem - 2
Percurso - 2
Hora Fim - 2
Km fim - 2
Material transportado - 2
Peso - 2
Hora Inicio - 3
Km origem - 3
Percurso - 3
Hora Fim - 3
Km fim - 3
Material transportado - 3
Peso - 3
Hora Inicio - 4
Km origem - 4
Percurso - 4
Hora Fim - 4
Km fim - 4
Material transportado - 4
Peso - 4
Hora Inicio - 5
Km origem - 5
Percurso - 5
Hora Fim - 5
Km fim - 5
Material transportado - 5
Peso - 5
Designação
Marca
Modelo
Nº Série
Ano de Fabricação
Verificar Sistemas de Segurança
Iluminação - Sistema de Aviso
Nivel de combustivel
Nível de oleo no mo

In [49]:
export2

['Identificação da Obra',
 'Estaca Inicial - Menor',
 'Fração Inicial (m)',
 'Estaca Final - Maior',
 'Fração Final (m)',
 'Comprimento',
 'Volume',
 'Observações',
 'Quantidade',
 'Viatura',
 'Placa',
 'Hora Inicio',
 'Percurso',
 'Hora Fim',
 'Material transportado',
 'Peso',
 'Diesel (L)',
 'Gasolina (L)',
 'Etanol (L)',
 'Óleo 68 (L)',
 '15W40 (L)',
 '85W90 (L)',
 'Graxa (kg)',
 'Horímetro Inicial',
 'Horímetro Final',
 'Horímetro',
 'Hora Inicio - 1',
 'Km origem - 1',
 'Percurso - 1',
 'Hora Fim - 1',
 'Km fim - 1',
 'Material transportado - 1',
 'Peso - 1',
 'Hora Inicio - 2',
 'Km origem - 2',
 'Percurso - 2',
 'Hora Fim - 2',
 'Km fim - 2',
 'Material transportado - 2',
 'Peso - 2',
 'Hora Inicio - 3',
 'Km origem - 3',
 'Percurso - 3',
 'Hora Fim - 3',
 'Km fim - 3',
 'Material transportado - 3',
 'Peso - 3',
 'Hora Inicio - 4',
 'Km origem - 4',
 'Percurso - 4',
 'Hora Fim - 4',
 'Km fim - 4',
 'Material transportado - 4',
 'Peso - 4',
 'Hora Inicio - 5',
 'Km origem - 5',
 

## Auxiliar montagem de permissões base

In [3]:
import json as js

HelpChat  sem query
QueuedPush  sem query


In [22]:
with open(moduloB + '.txt', 'w') as outfile:  
    js.dump(muduloAnalises, outfile, indent=4)

HelpChat  sem query
QueuedPush  sem query


In [32]:
comp = compare(a,c)

comp

{'comum': ['Contract',
  'HelpChat',
  'Resource',
  'ActionLog',
  'Equipment',
  'PDFImport',
  'Reporting',
  'ShapeFile',
  'TileLayer',
  'QueuedPush',
  'SubCompany',
  'UserInFirm',
  'FieldSurvey',
  'measurement',
  'ApprovalFlow',
  'ApprovalStep',
  'AccessRequest',
  'ExportRequest',
  'ContractService',
  'FieldSurveyRoad',
  'InspectorInFirm',
  'Advanced_options',
  'ReportingMessage',
  'AdditionalControl',
  'DailyReportConfig',
  'DailyReportExport',
  'DailyReportWorker',
  'ProcedureResource',
  'ApprovalTransition',
  'DailyReportVehicle',
  'ReportingAppConfig',
  'DailyReportRelation',
  'DailyReportResource',
  'MeasurementBulletin',
  'Measurement_service',
  'MultipleDailyReport',
  'DailyReportEquipment',
  'DailyReportSignaling',
  'FieldSurveySignature',
  'ServiceOrderResource',
  'ContractItemUnitPrice',
  'DailyReportOccurrence',
  'ContractItemPerformance',
  'DailyReportExternalTeam',
  'MeasurementBulletinExport',
  'ContractItemAdministration'],
 'ex

In [35]:
dict_replace={
 'job': 'Job',
 'firm': 'Firm',
 'goal': 'Goal',
 'road': 'Road',
 'user': 'User',
 'company': 'Company',
 'service': 'Service',
 'inventory': 'Inventory',
 'dashboard2': 'Dashboard2',
 'permission': 'Permission',
 'excel_import': 'Excel_import',
 'user_in_firm': 'User_in_firm',
 'service_usage': 'Service_usage',
 'reporting_file': 'Reporting_file',
 'occurrence_type': 'Occurrence_type',
 'user_in_company': 'User_in_company',
 'user_permission': 'User_permission',
 'ReportingDeadline': 'Reporting_deadline',
 'customization_menu': 'Customization_menu',
 'service_order_action_status': 'Service_order_action_status'
}

inv_map = {v: k for k, v in dict_replace.items()}
print(inv_map)

{'Job': 'job', 'Firm': 'firm', 'Goal': 'goal', 'Road': 'road', 'User': 'user', 'Company': 'company', 'Service': 'service', 'Inventory': 'inventory', 'Dashboard2': 'dashboard2', 'Permission': 'permission', 'Excel_import': 'excel_import', 'User_in_firm': 'user_in_firm', 'Service_usage': 'service_usage', 'Reporting_file': 'reporting_file', 'Occurrence_type': 'occurrence_type', 'User_in_company': 'user_in_company', 'User_permission': 'user_permission', 'Reporting_deadline': 'ReportingDeadline', 'Customization_menu': 'customization_menu', 'Service_order_action_status': 'service_order_action_status'}


In [36]:
permissaoCorrecao = c

for item in inv_map.keys():
    if item in permissaoCorrecao.keys():
        permissaoCorrecao[inv_map[item]] = permissaoCorrecao.pop(item)
        #pass

#print(permissaoCorrecao)
#print(list(permissaoCorrecao.keys()))
#print(inv_map)

In [43]:
print(list(c.keys()))

#comp = compare(c,a)
#print(comp['comum'])

['Contract', 'HelpChat', 'Resource', 'ActionLog', 'Equipment', 'PDFImport', 'Reporting', 'ShapeFile', 'TileLayer', 'QueuedPush', 'SubCompany', 'UserInFirm', 'FieldSurvey', 'measurement', 'ApprovalFlow', 'ApprovalStep', 'AccessRequest', 'ExportRequest', 'ContractService', 'FieldSurveyRoad', 'InspectorInFirm', 'Advanced_options', 'ReportingMessage', 'AdditionalControl', 'DailyReportConfig', 'DailyReportExport', 'DailyReportWorker', 'ProcedureResource', 'ApprovalTransition', 'DailyReportVehicle', 'ReportingAppConfig', 'DailyReportRelation', 'DailyReportResource', 'MeasurementBulletin', 'Measurement_service', 'MultipleDailyReport', 'DailyReportEquipment', 'DailyReportSignaling', 'FieldSurveySignature', 'ServiceOrderResource', 'ContractItemUnitPrice', 'DailyReportOccurrence', 'ContractItemPerformance', 'DailyReportExternalTeam', 'MeasurementBulletinExport', 'ContractItemAdministration', 'job', 'firm', 'road', 'user', 'company', 'service', 'inventory', 'dashboard2', 'permission', 'excel_impo

True

In [ ]:
recursos = ['ContractItemAdministration',
            'ContractItemPerformance',
            'ContractItemUnitPrice',
            'Contract',
            'ContractService',
            'Service'
            'Measurement'
            'MeasurementBulletinExport',
            'MeasurementService',
            'Resource']

rdo = ['DailyReportConfig',
       'DailyReportContractUsage',
       'DailyReportEquipment',
       'DailyReportExport',
       'DailyReportExternalTeam',
       'DailyReportOccurrence',
       'DailyReportRelation',
       'DailyReportResource',
       'DailyReportSignaling',
       'DailyReportVehicle',
       'DailyReportWorker',
       'MultipleDailyReport']

sisquali = ["quality_assay",
            "quality_sample",
            "quality_project",
            "quality_control_export"]

sisobras = ["construction",
            "construction_progress",
            "File"]

#sisoae = #????????

In [ ]:
  [f(x) if x is not None else '' for x in xs]